In [17]:
import json
import pandas as pd
import numpy as np

In [18]:
def read_table (filename):
    return pd.read_csv(filename, sep=";", index_col=0 )

In [ ]:

config_table = read_table("ConfigTable.csv")
mask_table = read_table("MaskTable.csv")

def toJSON (ASM_byte, first_byte):
    if (ASM_byte!=''):
        mask_byte = mask_table[first_byte[0]][first_byte[1]]
        ASM_byte = int (ASM_byte, 16)
        mask_byte = int (mask_byte, 16)
        return str(hex(ASM_byte & mask_byte))[2:]




print(config_table)
print(mask_table)




   0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F
0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
2  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
3  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
4  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
5  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
6  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
7  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
8  2  2  2  2  1  1  1  1  1  1  1  1  1  1  1  1
9  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
A  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
B  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
C  1  1  1  1  1  1  2  2  1  1  1  1  1  1  1  1
D  2  2  2  2  1  1  1  1  1  1  1  1  1  1  1  1
E  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
F  1  1  1  1  1  1  2  2  1  1  1  1  1  1  2  2
      0     1     2     3   4   5     6     7   8   9   A   B   C   D     E  \
0    F8    F8    F8    F8  F8  F8    E7    E7  F8  F8  F8  F8  F8  F8  

In [ ]:
import sys
import os
from tqdm import tqdm

main_dir = sys.argv[1]
output_dir = sys.argv[2]

for f in tqdm(os.listdir(main_dir), desc="Masking files", unit="file"):
    file_path = os.path.join(main_dir,f)
    with open (file_path, "r") as file:
        sorok = file.readlines()
        header =''
        commands = []
        ASM_byte = ''
        data = {"processes": []}
        for sor in sorok :
            prev_header = header
            if sor.__contains__('>:'):
                new_process = {}
                header = sor.split('<')[0]
                new_process['header'] = header
                commands = []
            elif (sor.__contains__(':')):
                sor = sor.split(':')[1]
                if (sor.__contains__(' ') and sor[0]!=' '):
                    sor = sor.split(' ')
                    first_byte = sor[0].strip('\t').upper()
                    bytes = config_table[first_byte[0]][first_byte[1]]
                    for x in range(bytes):
                        ASM_byte= ASM_byte+sor[x]
                
                    commands.append(toJSON(ASM_byte, first_byte))
                    ASM_byte=''  

            if (prev_header!= header and header!=''):
                new_process["commands"] = commands
                data["processes"].append(new_process)
            
        output_file_path = os.path.join(output_dir, f"{os.path.splitext(f)[0]}.json")
        with open(output_file_path, "w") as json_file:
            json.dump(data, json_file, indent=4)